In [1]:
from src.Model_Parameter import get_model_response

import re
import os
import csv
import json
from typing import List
import yaml
import openai
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange
from langchain import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
import requests
import xml.etree.ElementTree as ET
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import pandas as pd
data = pd.read_parquet('medalpaca_pubmed_summarization.parquet')
articles = pd.read_parquet('articles.parquet')
articles.head()

,article,abstract,record_id
0,gastric cancer is the second most frequent neo...,the typical symptoms of advanced cancer of the...,53
1,pathological gambling is defined in the curren...,abstractthe incidence of pathological gambling...,204
2,beyond its role of cellular energy currency an...,release and reception of extracellular atp by ...,217
3,generalized anxiety disorder ( gad ) is a chro...,objective : to elucidate the consequences of b...,244
4,sterilization in orthodontics has repeatedly b...,background : sterilization techniques could af...,260


In [3]:
merged_df = pd.merge(data, articles, on='record_id')
correct_summary = merged_df['mapped_ground_true'].tolist()
model_generated = merged_df['mapped_answer'].tolist()
articles = merged_df['article'].tolist()
print(len(correct_summary))
len(model_generated)

100


100

In [4]:
from rouge_score import rouge_scorer

In [5]:
# Summarize osmosis transcripts and evaluate
import time

# Start time
start_time = time.time()

#looping to go accross the entire data for evaluation

# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
for i, note in enumerate(correct_summary):
    
    # Evaluate BLEU score
    bleu_score = corpus_bleu([[correct_summary[i].split()]], [model_generated[i].split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = correct_summary[i]
    hypothesis = model_generated[i]
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

     # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    #print("Clinical Note:")
    #print(note)
    print("\nCorrect Summary:")
    print(correct_summary[i])
    print("\nGenerated Summary:")
    print(model_generated[i])
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")

    
end_time = time.time()

/Users/ritvikkhandelwal/opt/anaconda3/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/ritvikkhandelwal/opt/anaconda3/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)



Correct Summary:
during the last years , self - assembled organic nanostructures have been recognized as a proper fundament for several electrical and optical applications . in particular , phenylenes deposited on muscovite mica have turned out to be an outstanding material combination . 
 they tend to align parallel to each other forming needlelike structures . in that way 
 , they provide the key for macroscopic highly polarized emission , waveguiding , and lasing . 
 the resulting anisotropy has been interpreted so far by an induced dipole originating from the muscovite mica substrate . 
 based on a combined experimental and theoretical approach , we present an alternative growth model being able to explain molecular adsorption on sheet silicates in terms of moleculesurface interactions only . by a comprehensive comparison between experiments and simulations , 
 we demonstrate that geometrical changes in the substrate surface or molecule lead to different molecular adsorption geome

/Users/ritvikkhandelwal/opt/anaconda3/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)



Correct Summary:
objective : to elucidate the consequences of broadening dsm - iv criteria for generalized anxiety disorder ( gad ) , we examined prospectively the evolution of gad symptoms in two groups of patients ; one group diagnosed according to dsm - iv criteria and the other , according to broader criteria.method:multicentre , prospective and observational study conducted on outpatient psychiatric clinics . 
 patients were selected from october 2007 to january 2009 and diagnosed with gad according to dsm - iv criteria ( dsm - iv group ) or broader criteria . 
 broader criteria were considered 1-month of excessive or non - excessive worry and only 2 of the associated symptoms listed on dsm - iv for gad diagnosis . 
 socio - demographic data , medical history and functional outcome measures were collected three times during a 6-month period.results:3,549 patients were systematically recruited ; 1,815 patients in dsm - iv group ( dg ) and 1,264 in broad group ( bg ) ; 453 patients

Correct Summary:
diabetes is a world - wide epidemic associated with multiple environmental factors . 
 prolonged television viewing ( tv ) time has been related to increased risk of obesity and type 2 diabetes in several studies . 
 tv viewing has been positively associated with cardiovascular disease risk factors , lower energy expenditure , over - eating high - calorie and high - fat foods . 
 the objective of this study was to assess the associations of hours of tv viewing with dietary quality , obesity and physical activity for three ethnic minorities with and without type 2 diabetes . 
 diet quality and physical activity were inversely related to prolonged tv viewing . 
 african americans and participants with type 2 diabetes were more likely to watch more than 4 hours of tv per day as compared to their counterparts . diet quality was inversely associated with physical activity level . 
 future studies are needed to establish the risk factors of prolonged tv watching in adult pop


Correct Summary:
objectives : the aim of our study was to find out how much energy is applicable in second - generation dual source high - pitch computed tomography ( ct ) in imaging of the abdomen.materials and methods : we examined an upper abdominal phantom using a somatom definition flash ct - scanner ( siemens , forchheim , germany ) . the study protocol consisted of a scan - series at 100 kv and 120 kv . in each scan series 
 we started with a pitch of 3.2 and reduced it in steps of 0.2 , until a pitch of 1.6 was reached . 
 the current was adjusted to the maximum the scanner could achieve . 
 energy values , image noise , image quality , and radiation exposure were evaluated.results:for a pitch of 3.2 the maximum applicable current was 142 mas at 120 kv and in 100 kv the maximum applicable current was 114 mas . for conventional abdominal imaging , current levels of 200 to 260 mas are generally used . to achieve similar current levels , we had to decrease the pitch to 1.8 at 100


Correct Summary:
the role of cutaneous human papillomavirus ( hpv ) infection in the development of subsequent cutaneous squamous cell carcinoma ( scc ) is unknown . 
 pathologically confirmed cases of scc ( n = 150 ) enrolled in a previously conducted case - control study were included in a retrospective cohort study to examine the association of cutaneous hpv at the time of scc diagnosis with the risk of subsequent scc development . 
 data on hpv seropositivity , hpv dna in eyebrow hairs ( eb ) and scc tumors were available from the parent study . 
 incidence of subsequent scc was estimated using person - years of follow up . 
 cox proportional hazards ratios were estimated to evaluate the associations of both , hpv seropositivity and hpv dna positivity with subsequent scc . 
 the five year cumulative incidence of subsequent scc was 72% . 
 seropositivity to cutaneous hpv was not associated with the risk of subsequent scc ( hr = 0.83 , 95% ci = 0.411.67 ) . 
 any beta hpv infection 


Correct Summary:
objectivemany interventions to reduce allergen levels in the home are recommended to asthma and allergy patients . 
 one that is readily available and can be highly effective is the use of high performing filters in forced air ventilation systems.methodswe conducted a modeling analysis of the effectiveness of filter - based interventions in the home to reduce airborne asthma and allergy triggers . 
 this work used  each pass removal efficiency 
 applied to health - relevant size fractions of particles to assess filter performance . 
 we assessed effectiveness for key allergy and asthma triggers based on applicable particle sizes for cat allergen , indoor and outdoor sources of particles < 2.5  m in diameter ( pm2.5 ) , and airborne influenza and rhinovirus.resultsour analysis finds that higher performing filters can have significant impacts on indoor particle pollutant levels . 
 filters with removal efficiencies of > 70% for cat dander particles , fine particulate ma

In [6]:
# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

# Calculate and print average METEOR score
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
print("Average METEOR Score:", avg_meteor_score)

Average BLEU Score: 0.0025124497790341748
Average ROUGE-1 Score: 0.15631983253089732
Average ROUGE-2 Score: 0.03610656207286973
Average ROUGE-L Score: 0.09704180848277665
Average METEOR Score: 0.09378435106241159


In [7]:
print(len(articles))
print(len(correct_summary))
print(len(model_generated))
print(len(meteor_scores))

100
100
100
100


In [9]:
data = {
    'articles': articles,
    'correct_summary': correct_summary,
    'model_generated': model_generated,
    'meteor_scores': meteor_scores
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Exporting the DataFrame to an Excel file
df.to_excel('MedAlpaca_output.xlsx', index=False)